In [ ]:
#| default_exp handlers.tepco

# TEPCO 
> Data pipeline (handler) to convert TEPCO dataset ([Source](https://radioactivity.nsr.go.jp/ja/list/349/list-1.html)) to `NetCDF` format

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#| export
import pandas as pd
import re
import numpy as np
import fastcore.all as fc
from tqdm import tqdm
from collections import defaultdict

from marisco.callbacks import (
    Callback, 
    Transformer,
    EncodeTimeCB, 
    SanitizeLonLatCB,
    EncodeTimeCB, 
    )

from marisco.configs import cfg
from marisco.encoders import NetCDFEncoder

from marisco.metadata import (
    GlobAttrsFeeder, 
    BboxCB,
    TimeRangeCB,
    ZoteroCB, 
    KeyValuePairCB    
    )

from marisco.netcdf2csv import decode

## Configuration & file paths

In [ ]:
#| exports
fname_coastal_water = 'https://radioactivity.nra.go.jp/cont/en/results/sea/coastal_water.csv'
fname_clos1F = 'https://radioactivity.nra.go.jp/cont/en/results/sea/close1F_water.xlsx'
fname_iaea_orbs = 'https://raw.githubusercontent.com/RML-IAEA/iaea.orbs/refs/heads/main/src/iaea/orbs/stations/station_points.csv'

fname_out = '../../_data/output/tepco.nc'

## Load data

We here load the data from the [NRA (Nuclear Regulatory Authority)](https://radioactivity.nra.go.jp/en/results) website. For the moment, we only process radioactivity concentration data in the seawater around Fukushima Dai-ichi NPP [TEPCO] (`coastal_water.csv`) and in the `close1F_water.xlsx` file.

In near future, MARIS will provide a dedicated handler for all related [ALPS data](https://radioactivity.nra.go.jp/en/results#sec-12) including measurements not only provided by TEPCO but also MOE, NRA, MLITT and Fukushima Prefecture.



:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

The **coastal_water.csv** file contains two sections: the measurements and the locations. We identify below the line number where the locations begin. A single point of truth for the location of the stations would ease the processing in future.

:::

In [ ]:
#| exports
def find_location_section(df, 
                          col_idx=0,
                          pattern='Sampling point number'
                          ):
    "Find the line number where location data begins."
    mask = df.iloc[:, col_idx] == pattern
    indices = df[mask].index
    return indices[0] if len(indices) > 0 else -1

In [ ]:
#| eval: false
find_location_section(pd.read_csv(fname_coastal_water, low_memory=False))

np.int64(29252)

:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

Distinct parsing of the time from `coastal_water.csv` and `close1F_water.xlsx` files are required. Indeed:

- `coastal_water.csv` uses the format `YYYY/MM/DD` in the `Sampling  HH:MM` and 
- `close1F_water.xlsx` uses the format `YYYY-MM-DD HH:MM:SS`.

:::

In [ ]:
#| exports
def fix_sampling_time(x):
    if pd.isna(x): 
        return '00:00:00'
    else:
        hour, min =  x.split(':')[:2]
        return f"{hour if len(hour) == 2 else '0' + hour}:{min}:00"

In [ ]:
#| exports
def get_coastal_water_df(fname_coastal_water):
    "Get the measurements dataframe from the `coastal_water.csv` file."
    
    locs_idx = find_location_section(pd.read_csv(fname_coastal_water, 
                                      skiprows=0, low_memory=False))
    
    df = pd.read_csv(fname_coastal_water, skiprows=1, 
                     nrows=locs_idx - 1,
                     low_memory=False)
    df.dropna(subset=['Sampling point number'], inplace=True)
    df['Sampling time'] = df['Sampling time'].map(fix_sampling_time)
    
    df['TIME'] = df['Sampling date'].replace('-', '/') + ' ' + df['Sampling time']
    
    df = df.drop(columns=['Sampling date', 'Sampling time'])
    return df

In [ ]:
#| eval: false
df_coastal_water = get_coastal_water_df(fname_coastal_water)
df_coastal_water.tail()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME
29219,T-D5,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,6.2E+00,NaN,NaN,NaN,NaN,NaN,2025/7/17 07:56:00
29220,T-S8,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,6.8E+00,NaN,NaN,NaN,NaN,NaN,2025/7/18 05:34:00
29221,T-D5,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,7.9E+00,NaN,NaN,NaN,NaN,NaN,2025/7/21 08:05:00
29222,T-S3,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,7.3E+00,NaN,NaN,NaN,NaN,NaN,2025/7/22 05:54:00
29223,T-S4,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,7.4E+00,NaN,NaN,NaN,NaN,NaN,2025/7/22 06:17:00


In [ ]:
coi = [o for o in df_coastal_water.columns if "134Cs" in o]

df_coastal_water[coi + ['Sampling point number', 'TIME']].head(30)

,134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),Sampling point number,TIME
0,4.8E+01,9.2E+00,T-3,2011/3/21 23:15:00
1,3.1E+01,8.7E+00,T-4,2011/3/21 23:45:00
2,4.6E+01,1.4E+01,T-3,2011/3/22 14:28:00
3,3.9E+01,1.1E+01,T-4,2011/3/22 15:06:00
4,5.1E+01,2.0E+01,T-3,2011/3/23 13:51:00
5,3.3E+01,2.1E+01,T-4,2011/3/23 14:25:00
6,9.9E+01,3.8E+01,T-3,2011/3/24 09:30:00
7,3.5E+01,7.0E+00,T-4,2011/3/24 08:45:00
8,2.6E+01,7.4E+00,T-3,2011/3/25 10:00:00
9,2.0E+01,6.7E+00,T-4,2011/3/25 09:10:00


In [ ]:
coi

['134Cs radioactivity concentration (Bq/L)', '134Cs detection limit (Bq/L)']

In [ ]:
df_coastal_water.dropna(subset=coi, how='any')[coi + ['Sampling point number', 'TIME']]

,134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),Sampling point number,TIME
0,4.8E+01,9.2E+00,T-3,2011/3/21 23:15:00
1,3.1E+01,8.7E+00,T-4,2011/3/21 23:45:00
2,4.6E+01,1.4E+01,T-3,2011/3/22 14:28:00
3,3.9E+01,1.1E+01,T-4,2011/3/22 15:06:00
4,5.1E+01,2.0E+01,T-3,2011/3/23 13:51:00
...,...,...,...,...
29209,ND,1.1E-03,T-11,2025/6/27 09:41:00
29210,ND,1.3E-03,T-5,2025/6/27 08:09:00
29211,ND,1.2E-03,T-5,2025/6/27 08:09:00
29212,ND,1.1E-03,T-D9,2025/6/27 09:03:00


In [ ]:
mask = df_coastal_water['134Cs radioactivity concentration (Bq/L)'] == 'ND'
df_coastal_water[mask][coi + ['Sampling point number', 'TIME']]

,134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),Sampling point number,TIME
53,ND,NaN,5,2011/4/6 11:30:00
57,ND,NaN,8,2011/4/6 12:52:00
59,ND,NaN,10,2011/4/6 13:37:00
64,ND,NaN,T-7,2011/4/6 12:44:00
65,ND,NaN,T-7,2011/4/6 13:15:00
...,...,...,...,...
29209,ND,1.1E-03,T-11,2025/6/27 09:41:00
29210,ND,1.3E-03,T-5,2025/6/27 08:09:00
29211,ND,1.2E-03,T-5,2025/6/27 08:09:00
29212,ND,1.1E-03,T-D9,2025/6/27 09:03:00


In [ ]:
len(df_coastal_water)

29224

:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

Identification of the stations location requires three distinct files:

- the second section of the `coastal_water.csv` file
- the `R6zahyo.pdf` file further processed by [https://github.com/RML-IAEA/iaea.orbs](https://github.com/RML-IAEA/iaea.orbs)
- the second sections of all sheets of `close1F_water.xlsx` file
  
All files and sheets required to look up the location of the stations.

:::

In [ ]:
#| exports
def get_locs_coastal_water(fname_coastal_water):
    locs_idx = find_location_section(pd.read_csv(fname_coastal_water, 
                                      skiprows=0, low_memory=False))
    
    df = pd.read_csv(fname_coastal_water, skiprows=locs_idx+1, 
                     low_memory=False).iloc[:, :3]
    
    df.columns = ['STATION', 'LON', 'LAT']
    df.dropna(subset=['LAT'], inplace=True)
    df['org'] = 'coastal_seawater.csv'
    return df

In [ ]:
#| eval: false
df_locs_coastal_water = get_locs_coastal_water(fname_coastal_water)
print(f'Nb. of stations: {len(df_locs_coastal_water)}')
df_locs_coastal_water.head()

Nb. of stations: 48


,STATION,LON,LAT,org
0,T-0,37.42,141.04,coastal_seawater.csv
1,T-11,37.24,141.05,coastal_seawater.csv
2,T-12,37.15,141.04,coastal_seawater.csv
3,T-13-1,37.64,141.04,coastal_seawater.csv
4,T-14,37.55,141.06,coastal_seawater.csv


In [ ]:
#| eval: false
df_locs_coastal_water.STATION.unique()

array(['T-0', 'T-11', 'T-12', 'T-13-1', 'T-14', 'T-17-1', 'T-18', 'T-20',
       'T-22', 'T-3', 'T-4', 'T-4-1', 'T-4-2', 'T-5', 'T-6', 'T-7', 'T-A',
       'T-B', 'T-B1', 'T-B2', 'T-B3', 'T-B4', 'T-C', 'T-D', 'T-D1',
       'T-D5', 'T-D9', 'T-E', 'T-E1', 'T-Z', 'T-MG6', 'T-S1', 'T-S7',
       'T-H1', 'T-S2', 'T-S6', 'T-M10', 'T-MA', 'T-S3', 'T-S4', 'T-S8',
       'T-MG4', 'T-G4', 'T-MG5', 'T-MG1', 'T-MG0', 'T-MG3', 'T-MG2'],
      dtype=object)

:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

Data contained in the `close1F_water.xlsx` file are spread in several sheets (one per station). Each sheet further contains two sections: the measurements and the locations. 

For each sheet, we have to identify the line number where to split both measurements and the location. We then need to further iterate over all sheets to concatenate the results.

:::

In [ ]:
#| exports
def get_clos1F_df(fname_clos1F):
    "Get measurements dataframe from close1F_water.xlsx file and parse datetime."
    excel_file = pd.ExcelFile(fname_clos1F)
    dfs = {}
    
    for sheet_name in tqdm(excel_file.sheet_names):
        locs_idx = find_location_section(pd.read_excel(excel_file, 
                                                       sheet_name=sheet_name,
                                                       skiprows=1))
        df = pd.read_excel(excel_file, 
                   sheet_name=sheet_name, 
                   skiprows=1,
                   nrows=locs_idx-1)
        
        df.dropna(subset=['Sampling point number'], inplace=True)
        df['Sampling date'] = df['Sampling date']\
            .astype(str)\
            .apply(lambda x: x.split(' ')[0]\
            .replace('-', '/'))
            
        dfs[sheet_name] = df
    
    df = pd.concat(dfs.values(), ignore_index=True)
    df.dropna(subset=['Sampling date'], inplace=True)
    df['TIME'] = df['Sampling date'] + ' ' + df['Sampling time'].astype(str)
    df = df.drop(columns=['Sampling date', 'Sampling time'])
    return df

In [ ]:
#| eval: false
df_clos1F = get_clos1F_df(fname_clos1F)
df_clos1F.head()

100%|██████████| 11/11 [00:05<00:00,  2.05it/s]


,Sampling point number,134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),Total beta radioactivity concentration (Bq/L),Total beta detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),Collection layer of seawater,...,106Ru detection limit (Bq/L),60Co radioactivity concentration (Bq/L),60Co detection limit (Bq/L),95Zr radioactivity concentration (Bq/L),95Zr detection limit (Bq/L),99Mo radioactivity concentration (Bq/L),99Mo detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),TIME
0,T-0-1,ND,1.5,ND,1.4,ND,18.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/14 08:17:00
1,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,4.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/14 08:17:00
2,T-0-1,ND,1.1,ND,1.4,ND,20.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/21 08:09:00
3,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,ND,2.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/21 08:09:00
4,T-0-1,ND,0.66,ND,0.49,ND,17.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/27 08:14:00


In [ ]:
#| eval: false
df_clos1F['Sampling point number'].unique()

array(['T-0-1', 'T-0-1A', 'T-0-2', 'T-0-3', 'T-0-3A', 'T-1', 'T-2',
       'T-2-1', 'T-A1', 'T-A2', 'T-A3'], dtype=object)

In [ ]:
#| exports
def get_locs_clos1F(fname_clos1F):
    "Get locations dataframe from close1F_water.xlsx file from each sheets."
    excel_file = pd.ExcelFile(fname_clos1F)
    dfs = {}
    
    for sheet_name in tqdm(excel_file.sheet_names):
        locs_idx = find_location_section(pd.read_excel(excel_file, 
                                                       sheet_name=sheet_name,
                                                       skiprows=1))
        df = pd.read_excel(excel_file, 
                           sheet_name=sheet_name, 
                           skiprows=locs_idx+2)
            
        dfs[sheet_name] = df
    
    df = pd.concat(dfs.values(), ignore_index=True).iloc[:, :3]
    df.dropna(subset=['Sampling coordinate North latitude (Decimal)'], inplace=True)    
    df.columns = ['STATION', 'LON', 'LAT']
    df['org'] = 'close1F.csv'
    return df

In [ ]:
#| eval: false
df_locs_clos1F = get_locs_clos1F(fname_clos1F)
print(f'Nb. of stations: {len(df_locs_clos1F)}')
df_locs_clos1F.head()

100%|██████████| 11/11 [00:05<00:00,  1.99it/s]

Nb. of stations: 11


,STATION,LON,LAT,org
0,T-0-1,37.43,141.04,close1F.csv
11,T-0-1A,37.43,141.05,close1F.csv
22,T-0-2,37.42,141.05,close1F.csv
33,T-0-3,37.42,141.04,close1F.csv
44,T-0-3A,37.42,141.05,close1F.csv


:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

The `close1F_water.xlsx` file contains station locations that are not present in the `coastal_water.csv` dataset, as demonstrated in the comparison below:
:::

In [ ]:
#| eval: false
set(df_locs_clos1F.STATION) - set(df_locs_coastal_water.STATION)

{'T-0-1',
 'T-0-1A',
 'T-0-2',
 'T-0-3',
 'T-0-3A',
 'T-1',
 'T-2',
 'T-2-1',
 'T-A1',
 'T-A2',
 'T-A3'}

:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

In theory all locations are supposed to be provided in the [R6zahyo.pdf](https://radioactivity.nra.go.jp/cont/en/results/sea/R6zahyo.pdf) file. This file is further processed by https://github.com/RML-IAEA/iaea.orbs and the result is provided in the `station_points.csv` file. 

However, this file lacks complete coverage of locations referenced in both `coastal_water.csv` and `close1F_water.xlsx` files, while simultaneously containing additional locations not present in either (see below). A more standardized and comprehensive location reference system would significantly improve the efficiency and reliability of the data ingestion process.

:::

In [ ]:
#| exports
def get_locs_orbs(fname_iaea_orbs):
    df = pd.read_csv(fname_iaea_orbs)
    df.columns = ['org', 'STATION', 'LON', 'LAT']
    return df

In [ ]:
#| eval: false
df_locs_orbs = get_locs_orbs(fname_iaea_orbs)
df_locs_orbs.head()

,org,STATION,LON,LAT
0,MOE,E-31,141.727667,39.059167
1,MOE,E-32,141.635667,38.996000
2,MOE,E-37,141.948611,39.259167
3,MOE,E-38,141.755000,39.008333
4,MOE,E-39,141.766667,38.991667


In [ ]:
#| eval: False
set(df_locs_orbs.STATION) - (set(df_locs_clos1F.STATION) | set(df_locs_coastal_water.STATION))

{'C-P1',
 'C-P2',
 'C-P3',
 'C-P4',
 'C-P5',
 'C-P8',
 'E-31',
 'E-32',
 'E-37',
 'E-38',
 'E-39',
 'E-3A',
 'E-41',
 'E-42',
 'E-43',
 'E-44',
 'E-45',
 'E-46',
 'E-47',
 'E-48',
 'E-49',
 'E-4A',
 'E-4B',
 'E-4C',
 'E-4F',
 'E-4G',
 'E-4H',
 'E-4J',
 'E-4K',
 'E-4L',
 'E-4M',
 'E-71',
 'E-72',
 'E-73',
 'E-74',
 'E-75',
 'E-76',
 'E-77',
 'E-78',
 'E-79',
 'E-7A',
 'E-7B',
 'E-7C',
 'E-7D',
 'E-7F',
 'E-7G',
 'E-7H',
 'E-7I',
 'E-7J',
 'E-7K',
 'E-7L',
 'E-81',
 'E-82',
 'E-83',
 'E-84',
 'E-85',
 'E-S1',
 'E-S10',
 'E-S13',
 'E-S14',
 'E-S15',
 'E-S17',
 'E-S18',
 'E-S19',
 'E-S20',
 'E-S21',
 'E-S22',
 'E-S23',
 'E-S24',
 'E-S25',
 'E-S26',
 'E-S27',
 'E-S28',
 'E-S29',
 'E-S3',
 'E-S30',
 'E-S31',
 'E-S32',
 'E-S33',
 'E-S34',
 'E-S35',
 'E-S36',
 'E-S4',
 'E-S5',
 'E-T1',
 'E-T2',
 'E-T3',
 'E-T4',
 'E-T5',
 'E-T6',
 'E-T7',
 'E-T8',
 'F-P01',
 'F-P02',
 'F-P03',
 'F-P04',
 'F-P05',
 'F-P06',
 'F-P07',
 'F-P08',
 'F-P09',
 'F-P10',
 'F-P11',
 'F-P12',
 'F-P13',
 'F-P14',
 'F-P15'

In [ ]:
#| exports
def concat_locs(dfs):
    "Concatenate and drop duplicates from coastal_seawater.csv and iaea_orbs.csv (kept)"
    df = pd.concat(dfs)
    # Group by org to be used for sorting
    df['org_grp'] = df['org'].apply(
        lambda x: 1 if x == 'coastal_seawater.csv' else 2 if x == 'close1F.csv' else 0)
    df.sort_values('org_grp', ascending=True, inplace=True)
    # Drop duplicates and keep orbs data first
    df.drop_duplicates(subset='STATION', keep='first', inplace=True)
    df.drop(columns=['org_grp'], inplace=True)
    df.sort_values('STATION', ascending=True, inplace=True)
    return df

In [ ]:
#| eval: false
df_locs = concat_locs([df_locs_clos1F, df_locs_coastal_water, df_locs_orbs])
df_locs.head()

,STATION,LON,LAT,org
214,C-P1,139.863333,35.425000,NRA
215,C-P2,139.863333,35.401667,NRA
216,C-P3,139.881667,35.370000,NRA
217,C-P4,139.846667,35.356667,NRA
218,C-P5,139.800000,35.343333,NRA


In [ ]:
#| exports
def align_dfs(df_from, df_to):
    "Align columns structure of df_from to df_to."
    df = defaultdict()    
    for c in df_to.columns:
        df[c] = df_from[c].values if c in df_from.columns else np.nan
    return pd.DataFrame(df)

In [ ]:
# | eval: false
align_dfs(df_clos1F, df_coastal_water).head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME
0,T-0-1,NaN,NaN,NaN,ND,1.5,ND,1.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/14 08:17:00
1,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.7,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/14 08:17:00
2,T-0-1,NaN,NaN,NaN,ND,1.1,ND,1.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/21 08:09:00
3,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ND,2.9,NaN,NaN,NaN,NaN,NaN,2013/08/21 08:09:00
4,T-0-1,NaN,NaN,NaN,ND,0.66,ND,0.49,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013/08/27 08:14:00


In [ ]:
#| exports
def concat_dfs(df_coastal_water, df_clos1F):
    "Concatenate and drop duplicates from coastal_seawater.csv and close1F_water.xlsx (kept)"
    df_clos1F = align_dfs(df_clos1F, df_coastal_water)
    df = pd.concat([df_coastal_water, df_clos1F])
    return df

In [ ]:
#| eval: false
df_meas = concat_dfs(df_coastal_water, df_clos1F)
df_meas.head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,NaN,NaN,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00


In [ ]:
#| exports
def georef_data(df_meas, df_locs):
    "Georeference measurements dataframe using locations dataframe."
    assert "Sampling point number" in df_meas.columns and "STATION" in df_locs.columns
    return pd.merge(df_meas, df_locs, how="inner", 
                    left_on='Sampling point number', right_on='STATION')

In [ ]:
#| eval: false
df_meas_georef = georef_data(df_meas, df_locs)
df_meas_georef.head()

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,STATION,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


In [ ]:
#| exports
def load_data(fname_coastal_water, fname_clos1F, fname_iaea_orbs):
    "Load, align and georeference TEPCO data"
    df_locs = concat_locs(
        [get_locs_coastal_water(fname_coastal_water), 
         get_locs_clos1F(fname_clos1F),
         get_locs_orbs(fname_iaea_orbs)])
    df_meas = concat_dfs(get_coastal_water_df(fname_coastal_water), get_clos1F_df(fname_clos1F))
    df_meas.dropna(subset=['Sampling point number'], inplace=True)
    return {'SEAWATER': georef_data(df_meas, df_locs)}

In [ ]:
#| eval: false
dfs = load_data(fname_coastal_water, fname_clos1F, fname_iaea_orbs)
dfs['SEAWATER'].head()

100%|██████████| 11/11 [00:05<00:00,  2.08it/s]


,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,STATION,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


In [ ]:
#| eval: false
print(f"# of cols, rows: {dfs['SEAWATER'].shape}")
dfs['SEAWATER'].head()

# of cols, rows: (49863, 53)


,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,STATION,LON,LAT,org
0,T-3,NaN,1.1E+03,1.3E+01,4.8E+01,9.2E+00,5.3E+01,8.8E+00,1.6E+02,44.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:15:00,T-3,141.026389,37.322222,TEPCO
1,T-4,NaN,6.6E+02,1.2E+01,3.1E+01,8.7E+00,3.3E+01,8.3E+00,1.2E+02,41.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/21 23:45:00,T-4,141.013889,37.241667,TEPCO
2,T-3,NaN,1.1E+03,2.0E+01,4.6E+01,1.4E+01,4.0E+01,1.4E+01,ND,88.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 14:28:00,T-3,141.026389,37.322222,TEPCO
3,T-4,NaN,6.7E+02,1.9E+01,3.9E+01,1.1E+01,4.4E+01,1.1E+01,ND,79.0,...,NaN,NaN,NaN,NaN,NaN,2011/3/22 15:06:00,T-4,141.013889,37.241667,TEPCO
4,T-3,NaN,7.4E+02,2.7E+01,5.1E+01,2.0E+01,5.5E+01,2.0E+01,2.0E+02,58.0,...,NaN,NaN,34.0,25.0,NaN,2011/3/23 13:51:00,T-3,141.026389,37.322222,TEPCO


In [ ]:
#| eval: false
dfs['SEAWATER'].STATION.unique()

array(['T-3', 'T-4', 'T-5', 'T-7', 'T-11', 'T-12', 'T-14', 'T-18', 'T-20',
       'T-22', 'T-MA', 'T-M10', 'T-A', 'T-D', 'T-E', 'T-B', 'T-C',
       'T-MG1', 'T-MG2', 'T-MG3', 'T-MG4', 'T-MG5', 'T-MG6', 'T-D1',
       'T-D5', 'T-D9', 'T-E1', 'T-G4', 'T-H1', 'T-S5', 'T-S6', 'T-17-1',
       'T-B3', 'T-13-1', 'T-S3', 'T-S4', 'T-B4', 'T-S1', 'T-S2', 'T-MG0',
       'T-Z', 'T-B1', 'T-B2', 'T-S7', 'T-S8', 'T-0', 'T-4-1', 'T-4-2',
       'T-6', 'T-0-1', 'T-0-1A', 'T-0-2', 'T-0-3', 'T-0-3A', 'T-1', 'T-2',
       'T-2-1', 'T-A1', 'T-A2', 'T-A3'], dtype=object)

In [ ]:
np.sum(dfs['SEAWATER'] == "ND")

Sampling point number                                 0
Collection layer of seawater                          0
131I radioactivity concentration (Bq/L)            8642
131I detection limit (Bq/L)                           0
134Cs radioactivity concentration (Bq/L)          30967
134Cs detection limit (Bq/L)                          0
137Cs radioactivity concentration (Bq/L)          17232
137Cs detection limit (Bq/L)                          0
132I radioactivity concentration (Bq/L)               3
132I detection limit (Bq/L)                           0
132Te radioactivity concentration (Bq/L)              0
132Te detection limit (Bq/L)                          0
136Cs radioactivity concentration (Bq/L)              2
136Cs detection limit (Bq/L)                          0
140La radioactivity concentration (Bq/L)              0
140La detection limit (Bq/L)                          0
89Sr radioactivity concentration (Bq/L)             101
89Sr detection limit (Bq/L)                     

In [ ]:
dfs['SEAWATER'][['TIME', '134Cs radioactivity concentration (Bq/L)', '134Cs detection limit (Bq/L)']]

,TIME,134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L)
0,2011/3/21 23:15:00,4.8E+01,9.2E+00
1,2011/3/21 23:45:00,3.1E+01,8.7E+00
2,2011/3/22 14:28:00,4.6E+01,1.4E+01
3,2011/3/22 15:06:00,3.9E+01,1.1E+01
4,2011/3/23 13:51:00,5.1E+01,2.0E+01
...,...,...,...
49858,2025/06/30 08:05,ND,0.4
49859,2025/07/07 08:36,ND,0.37
49860,2025/07/17 08:11,ND,0.29
49861,2025/07/21 08:20,ND,0.36


## Remove 約 (about) character
    

:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

We systematically remove the `約` character. Please confirm that this is the correct way to handle this. We could imagine that mentioning uncertainty would be less ambiguous in future.

:::

In [ ]:
#| exports
class RemoveJapanaseCharCB(Callback):
    "Remove 約 (about) char"
    def _transform_if_about(self, value, about_char='約'):
        if pd.isna(value): return value
        return (value.replace(about_char, '') if str(value).count(about_char) != 0 
                else value)
    
    def __call__(self, tfm): 
        for k in tfm.dfs.keys():
            cols_rdn = [c for c in tfm.dfs[k].columns if ('(Bq/L)' in c) and (tfm.dfs[k][c].dtype == 'object')]
            tfm.dfs[k][cols_rdn] = tfm.dfs[k][cols_rdn].map(self._transform_if_about)

In [ ]:
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB()])

tfm()['SEAWATER'].sample(10)

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,STATION,LON,LAT,org
21965,T-MG4,下層,NaN,NaN,ND,1.2E-03,2.7E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022/9/13 14:02:00,T-MG4,141.133333,38.250000,TEPCO
32724,T-0-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022/04/20 07:35:00,T-0-3,141.040278,37.416111,TEPCO
10049,T-MG1,下層,NaN,NaN,1.8E-03,NaN,9.8E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2015/10/14 10:55:00,T-MG1,141.283333,38.333333,TEPCO
8337,T-4,上層,NaN,NaN,2.7E-02,NaN,9.0E-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2014/12/9 16:00:00,T-4,141.013889,37.241667,TEPCO
44057,T-2,上層,NaN,NaN,ND,0.58,ND,0.69,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2020/09/10 06:55:00,T-2,141.033611,37.415833,TEPCO
23250,T-D1,下層,NaN,NaN,ND,1.2E-03,3.4E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023/6/28 08:22:00,T-D1,141.072222,37.500000,TEPCO
28059,T-0-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2020/12/07 07:14:00,T-0-1,141.040278,37.430556,TEPCO
12321,T-MG5,上層,NaN,NaN,ND,1.4E-03,3.7E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2017/1/13 09:26:00,T-MG5,141.250000,38.166667,TEPCO
19891,T-MA,上層,NaN,NaN,ND,1.1E-03,2.9E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2021/7/6 04:24:00,T-MA,141.083333,37.750000,TEPCO
39626,T-1,上層,NaN,NaN,ND,0.71,ND,0.59,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2022/02/09 09:00:00,T-1,141.034444,37.431111,TEPCO


## Fix values range string

:::{.callout-important}
## FEEDBACK TO DATA PROVIDER

Value ranges are provided as strings (e.g '4.0E+00<&<8.0E+00' or '1.0～2.7'). We replace them by their mean. Please confirm that this is the correct way to handle this. Again, mentioning uncertainty would be less ambiguous in future.

:::

In [ ]:
#| exports
class FixRangeValueStringCB(Callback):
    "Replace range values (e.g '4.0E+00<&<8.0E+00' or '1.0～2.7') by their mean"
    
    def _extract_and_calculate_mean(self, s):
        # For scientific notation ranges
        float_strings = re.findall(r"[+-]?\d+\.?\d*E?[+-]?\d*", s)
        if float_strings:
            float_numbers = np.array(float_strings, dtype=float)
            return float_numbers.mean()
        return s
    
    def _transform_if_range(self, value):
        if pd.isna(value): 
            return value
        value = str(value)
        # Check for both range patterns
        if '<&<' in value or '～' in value:
            return self._extract_and_calculate_mean(value)
        return value

    def __call__(self, tfm): 
        for k in tfm.dfs.keys():
            cols_rdn = [c for c in tfm.dfs[k].columns 
                       if ('(Bq/L)' in c) and (tfm.dfs[k][c].dtype == 'object')]
            # tfm.dfs[k][cols_rdn] = tfm.dfs[k][cols_rdn].map(self._transform_if_range).astype(float)
            tfm.dfs[k][cols_rdn] = tfm.dfs[k][cols_rdn].map(self._transform_if_range)

In [ ]:
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB()
    ])

df_test = tfm()['SEAWATER']
df_test.sample(10)

,Sampling point number,Collection layer of seawater,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),132I radioactivity concentration (Bq/L),132I detection limit (Bq/L),...,125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L),Unnamed: 49,TIME,STATION,LON,LAT,org
12732,T-11,上層,NaN,NaN,ND,1.4E-03,1.1E-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2017/4/17 06:58:00,T-11,141.047222,37.241667,TEPCO
14319,T-D1,下層,NaN,NaN,ND,1.4E-03,3.9E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2018/3/19 08:02:00,T-D1,141.072222,37.500000,TEPCO
4247,T-D9,上層,NaN,NaN,2.4E-02,NaN,3.8E-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2012/11/28 09:05:00,T-D9,141.072167,37.333333,TEPCO
35043,T-1,上層,ND,0.56,1.7,NaN,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2012/04/04 08:45:00,T-1,141.034444,37.431111,TEPCO
44021,T-2,上層,NaN,NaN,ND,0.64,ND,0.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2020/08/16 06:45:00,T-2,141.033611,37.415833,TEPCO
5667,T-MG3,上層,NaN,NaN,ND,2.2E-03,4.0E-03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2013/8/20 08:54:00,T-MG3,141.583333,38.233333,TEPCO
48514,T-A1,上層,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2023/11/16 07:12:00,T-A1,141.050761,37.440794,TEPCO
5976,T-D9,下層,NaN,NaN,6.4E-03,NaN,1.5E-02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2013/10/8 10:22:00,T-D9,141.072167,37.333333,TEPCO
12143,T-A,上層,NaN,NaN,ND,9.8E-01,ND,1.1E+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016/12/5 10:53:00,T-A,140.763889,36.713889,TEPCO
11489,T-D,下層,NaN,NaN,ND,1.0E+00,ND,1.2E+00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2016/7/13 13:20:00,T-D,140.702222,35.987500,TEPCO


## Select columns of interest

We select the columns of interest and in particular the elements of interest, in our case radionuclides.

In [ ]:
#| exports
common_coi = ['LON', 'LAT', 'TIME', 'STATION']
nuclides_pattern = '(Bq/L)'

In [ ]:
#| exports
class SelectColsOfInterestCB(Callback):
    "Select columns of interest."
    def __init__(self, common_coi, nuclides_pattern): fc.store_attr()
    def __call__(self, tfm):
        nuc_of_interest = [c for c in tfm.dfs['SEAWATER'].columns if nuclides_pattern in c]
        tfm.dfs['SEAWATER'] = tfm.dfs['SEAWATER'][self.common_coi + nuc_of_interest]

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern)
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

,LON,LAT,TIME,STATION,131I radioactivity concentration (Bq/L),131I detection limit (Bq/L),134Cs radioactivity concentration (Bq/L),134Cs detection limit (Bq/L),137Cs radioactivity concentration (Bq/L),137Cs detection limit (Bq/L),...,144Ce radioactivity concentration (Bq/L),144Ce detection limit (Bq/L),54Mn radioactivity concentration (Bq/L),54Mn detection limit (Bq/L),3H radioactivity concentration (Bq/L),3H detection limit (Bq/L),125Sb radioactivity concentration (Bq/L),125Sb detection limit (Bq/L),105Ru radioactivity concentration (Bq/L),105Ru detection limit (Bq/L)
25819,141.042500,37.640833,2024/10/25 10:37:00,T-13-1,NaN,NaN,ND,1.4E-03,5.2E-03,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22884,141.072222,37.500000,2023/4/5 07:45:00,T-D1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46683,37.410000,141.030000,2013/02/10 07:00:00,T-2-1,ND,0.44,ND,1.1,ND,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,ND,NaN,NaN,NaN
35800,141.034444,37.431111,2013/12/30 06:45:00,T-1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ND,1.5,NaN,NaN,NaN,NaN
25861,141.042222,37.584722,2024/10/28 09:11:00,T-S1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.2E-01,NaN,NaN,NaN,NaN,NaN


## Reshape: wide to long

So that we can extract information such as nuclide name, unit, derived quantities such as uncertainty, detection limit, ...

In [ ]:
#| exports
class WideToLongCB(Callback):
    """
    Get TEPCO nuclide names as values not column names 
    to extract contained information (nuclide name, unc, dl, ...).
    """
    def __init__(self, id_vars=['LON', 'LAT', 'TIME', 'STATION']): 
        fc.store_attr()
        
    def __call__(self, tfm): 
        tfm.dfs['SEAWATER'] = pd.melt(tfm.dfs['SEAWATER'], id_vars=self.id_vars)
#| eval: false

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.head()

,LON,LAT,TIME,STATION,variable,value
0,141.026389,37.322222,2011/3/21 23:15:00,T-3,131I radioactivity concentration (Bq/L),1.1E+03
1,141.013889,37.241667,2011/3/21 23:45:00,T-4,131I radioactivity concentration (Bq/L),6.6E+02
2,141.026389,37.322222,2011/3/22 14:28:00,T-3,131I radioactivity concentration (Bq/L),1.1E+03
3,141.013889,37.241667,2011/3/22 15:06:00,T-4,131I radioactivity concentration (Bq/L),6.7E+02
4,141.026389,37.322222,2011/3/23 13:51:00,T-3,131I radioactivity concentration (Bq/L),7.4E+02


## Extract

Nulide name, dl, unc, ... are extracted from column names as embedded in TEPCO data source.

### Nuclide name

In [ ]:
#| exports
def extract_nuclide(text: str) -> str:
    "Extract the nuclide identifier from a measurement variable name using regex."
    pattern = r'^(Total\s+(?:alpha|beta)|[^\s]+)'
    match = re.match(pattern, text, re.IGNORECASE)
    return match.group(1) if match else text 

For instance:

In [ ]:
print(extract_nuclide("Total alpha radioactivity concentration (Bq/L)"))
print(extract_nuclide("131I radioactivity concentration (Bq/L)"))

Total alpha
131I


In [ ]:
#| exports
class ExtractNuclideNameCB(Callback):
    "Extract nuclide name from TEPCO data."
    def __init__(self, src_col='variable', dest_col='NUCLIDE'): fc.store_attr()
    def __call__(self, tfm): 
        tfm.dfs['SEAWATER'][self.dest_col] = tfm.dfs['SEAWATER'][self.src_col].map(extract_nuclide)

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

,LON,LAT,TIME,STATION,variable,value,NUCLIDE
397533,141.050761,37.440794,2023/10/25 07:34:00,T-A1,132I detection limit (Bq/L),NaN,132I
1164208,141.200000,37.416667,2020/1/6 08:20:00,T-5,Total alpha radioactivity concentration (Bq/L),NaN,Total alpha
572215,141.072167,37.333333,2023/10/4 08:54:00,T-D9,136Cs detection limit (Bq/L),NaN,136Cs
2223811,141.046667,37.430556,2023/08/30 07:56:00,T-0-1A,105Ru detection limit (Bq/L),NaN,105Ru
2222752,141.040278,37.430556,2025/05/01 06:57,T-0-1,105Ru detection limit (Bq/L),NaN,105Ru


### Unit

In [ ]:
#| exports
class ExtractUnitCB(Callback):
    "Extract unit from TEPCO data."
    def __init__(self, src_col='variable', dest_col='UNIT'): fc.store_attr()
    def __call__(self, tfm): 
        tfm.dfs['SEAWATER'][self.dest_col] = tfm.dfs['SEAWATER'][self.src_col].str.extract(r'\((.*?)\)')

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

,LON,LAT,TIME,STATION,variable,value,NUCLIDE,UNIT
26744,141.072167,37.333333,2025/3/24 08:23:00,T-D9,131I radioactivity concentration (Bq/L),NaN,131I,Bq/L
1322632,141.042222,37.584722,2024/12/18 09:40:00,T-S1,Total beta detection limit (Bq/L),NaN,Total beta,Bq/L
306035,141.000000,38.083333,2014/3/18 10:16:00,T-MG6,132I radioactivity concentration (Bq/L),NaN,132I,Bq/L
423205,141.233333,37.516667,2024/1/30 07:45:00,T-B2,132Te radioactivity concentration (Bq/L),NaN,132Te,Bq/L
1589882,141.033611,37.415833,2020/11/01 06:50:00,T-2,58Co radioactivity concentration (Bq/L),NaN,58Co,Bq/L


### Value type
Is it a measurement or derived detection such as detection limit or uncertainty?

In [ ]:
#| exports
class ExtractValueTypeCB(Callback):
    "Extract value type from TEPCO data."
    def __init__(self, src_col='variable', dest_col='type'): fc.store_attr()
    def __call__(self, tfm): 
        tfm.dfs['SEAWATER'][self.dest_col] = np.select(
            [
                tfm.dfs['SEAWATER'][self.src_col].str.contains('detection limit', case=False),
                tfm.dfs['SEAWATER'][self.src_col].str.contains('statistical error', case=False)],
            ['DL', 'UNC'],
            default='VALUE'
        )

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

,LON,LAT,TIME,STATION,variable,value,NUCLIDE,UNIT,type
895288,37.410000,141.030000,2015/03/03 06:00:00,T-2-1,90Sr detection limit (Bq/L),NaN,90Sr,Bq/L,DL
120105,141.283333,38.333333,2021/10/18 09:21:00,T-MG1,134Cs radioactivity concentration (Bq/L),ND,134Cs,Bq/L,VALUE
187490,141.034444,37.431111,2018/07/06 07:42:00,T-1,134Cs detection limit (Bq/L),0.58,134Cs,Bq/L,DL
1970516,141.042222,37.584722,2024/10/28 09:11:00,T-S1,3H radioactivity concentration (Bq/L),NaN,3H,Bq/L,VALUE
276978,141.040278,37.430556,2017/02/13 07:40:00,T-0-1,137Cs detection limit (Bq/L),NaN,137Cs,Bq/L,DL


## Reshape: long to wide
Send `type` column to columns names (`VALUE`, `DL`, `UNC`)


In [ ]:
#| exports
class LongToWideCB(Callback):
    "Reshape: long to wide"
    def __init__(self, src_col='variable', dest_col='type'): fc.store_attr()
    def __call__(self, tfm): 
        tfm.dfs['SEAWATER'] = pd.pivot_table(
            tfm.dfs['SEAWATER'],
            values='value',
            index=['LON', 'LAT', 'TIME', 'STATION', 'NUCLIDE', 'UNIT'],
            columns='type',
            aggfunc='first'
        ).reset_index()
        tfm.dfs['SEAWATER'].reset_index(inplace=True)
        tfm.dfs['SEAWATER'].rename(columns={'index': 'SMP_ID'}, inplace=True)

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE
24977,24977,141.033611,37.415833,2018/04/02 07:55:00,T-2,134Cs,Bq/L,0.68,NaN,ND
88334,88334,141.250000,38.166667,2018/11/1 09:24:00,T-MG5,134Cs,Bq/L,1.4E-03,NaN,ND
12161,12161,141.000000,36.966667,2017/2/25 06:29:00,T-20,137Cs,Bq/L,NaN,NaN,4.0E-03
50721,50721,141.040278,37.416111,2014/07/22 10:17:00,T-0-3,3H,Bq/L,1.7,NaN,ND
69305,69305,141.050761,37.424686,2022/12/26 07:24:00,T-A2,137Cs,Bq/L,0.25,NaN,ND


In [ ]:
df_test[df_test.VALUE == 'ND'].groupby('NUCLIDE').size().sort_values(ascending=False)

NUCLIDE
134Cs          25186
137Cs          16447
3H              8976
131I            7958
Total beta      4913
Total alpha      979
125Sb            647
90Sr             342
238Pu            308
239Pu+240Pu      231
89Sr             100
144Ce              9
54Mn               9
60Co               9
58Co               3
132I               3
136Cs              2
dtype: int64

In [ ]:
df_test[df_test.VALUE == 'ND']


type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE
0,0,37.210000,141.01,2012/10/16 07:25:00,T-4-1,131I,Bq/L,1.3E-01,NaN,ND
1,1,37.210000,141.01,2012/10/16 07:25:00,T-4-1,134Cs,Bq/L,1.9E-01,NaN,ND
2,2,37.210000,141.01,2012/10/16 07:25:00,T-4-1,137Cs,Bq/L,2.7E-01,NaN,ND
3,3,37.210000,141.01,2012/10/2 07:30:00,T-4-1,131I,Bq/L,1.1E-01,NaN,ND
4,4,37.210000,141.01,2012/10/2 07:30:00,T-4-1,134Cs,Bq/L,2.2E-01,NaN,ND
...,...,...,...,...,...,...,...,...,...,...
93158,93158,141.666667,38.30,2025/4/8 08:20:00,T-MG2,134Cs,Bq/L,1.3E-03,NaN,ND
93160,93160,141.666667,38.30,2025/5/13 07:36:00,T-MG2,134Cs,Bq/L,1.2E-03,NaN,ND
93162,93162,141.666667,38.30,2025/5/13 07:50:00,T-MG2,134Cs,Bq/L,8.7E-04,NaN,ND
93164,93164,141.666667,38.30,2025/6/3 08:15:00,T-MG2,134Cs,Bq/L,1.1E-03,NaN,ND


In [ ]:
df_test.VALUE == 'ND'

0         True
1         True
2         True
3         True
4         True
         ...  
93163    False
93164     True
93165    False
93166     True
93167    False
Name: VALUE, Length: 93168, dtype: bool

## Remap `UNIT` name to MARIS nomenclature

Data are reported in `Bq/L` but MARIS uses `Bq/m3` instead. So we assign it to MARIS `unit_id` = 3 (Bq/L).
Later in the processing pipeline, we will convert the values from Bq/L to Bq/m3 by multiplying VALUE, DL, and DLV by 1000.

In [ ]:
#| exports
unit_mapping = {'Bq/L': 1}

In [ ]:
#| exports
class RemapUnitNameCB(Callback):
    """
    Remap `UNIT` name to MARIS id.
    """
    def __init__(self, unit_mapping): fc.store_attr()
    def __call__(self, tfm):
        tfm.dfs['SEAWATER']['UNIT'] = tfm.dfs['SEAWATER']['UNIT'].map(self.unit_mapping)


In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping)
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE
73748,73748,141.072167,37.333333,2019/9/4 08:14:00,T-D9,90Sr,1,NaN,NaN,1.4E-03
49030,49030,141.034444,37.431111,2024/11/22 07:40:00,T-1,137Cs,1,0.62,NaN,ND
60948,60948,141.046667,37.416111,2024/11/11 08:09:00,T-0-3A,137Cs,1,0.28,NaN,ND
32329,32329,141.033611,37.415833,2023/12/25 07:09:00,T-2,137Cs,1,0.85,NaN,ND
66933,66933,141.047222,37.241667,2012/3/17 07:40:00,T-11,131I,1,7.0E-01,NaN,ND


## Remap `NUCLIDE` name to MARIS nomenclature

In [ ]:
#| exports
nuclide_mapping = {
    '131I': 29,
    '134Cs': 31,
    '137Cs': 33,
    '125Sb': 24,
    'Total beta': 103,
    '238Pu': 67,
    '239Pu+240Pu': 77,
    '3H': 1,
    '89Sr': 11,
    '90Sr': 12,
    'Total alpha': 104,
    '132I': 100,
    '136Cs': 102,
    '58Co': 8,
    '105Ru': 97,
    '106Ru': 17,
    '140La': 35,
    '140Ba': 34,
    '132Te': 99,
    '60Co': 9,
    '144Ce': 37,
    '54Mn': 6
}

In [ ]:
#| exports
class RemapNuclideNameCB(Callback):
    "Remap `NUCLIDE` name to MARIS id."
    def __init__(self, nuclide_mapping): fc.store_attr()
    def __call__(self, tfm):
        tfm.dfs['SEAWATER']['NUCLIDE'] = tfm.dfs['SEAWATER']['NUCLIDE'].map(self.nuclide_mapping)

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping)
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE
26104,26104,141.033611,37.415833,2018/12/25 07:22:00,T-2,29,1,0.67,NaN,ND
7562,7562,140.665556,36.506389,2017/11/15 08:10:00,T-B,31,1,1.1E+00,NaN,ND
28227,28227,141.033611,37.415833,2020/08/29 06:45:00,T-2,31,1,0.75,NaN,ND
50119,50119,141.037500,37.150000,2023/2/22 05:37:00,T-12,1,1,3.5E-01,NaN,ND
48642,48642,141.034444,37.431111,2024/06/17 07:20:00,T-1,33,1,0.69,NaN,ND


In [ ]:
df_test.dropna(subset=['DL', 'VALUE'], how='any')

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE
0,0,37.210000,141.01,2012/10/16 07:25:00,T-4-1,29,1,1.3E-01,NaN,ND
1,1,37.210000,141.01,2012/10/16 07:25:00,T-4-1,31,1,1.9E-01,NaN,ND
2,2,37.210000,141.01,2012/10/16 07:25:00,T-4-1,33,1,2.7E-01,NaN,ND
3,3,37.210000,141.01,2012/10/2 07:30:00,T-4-1,29,1,1.1E-01,NaN,ND
4,4,37.210000,141.01,2012/10/2 07:30:00,T-4-1,31,1,2.2E-01,NaN,ND
...,...,...,...,...,...,...,...,...,...,...
93158,93158,141.666667,38.30,2025/4/8 08:20:00,T-MG2,31,1,1.3E-03,NaN,ND
93160,93160,141.666667,38.30,2025/5/13 07:36:00,T-MG2,31,1,1.2E-03,NaN,ND
93162,93162,141.666667,38.30,2025/5/13 07:50:00,T-MG2,31,1,8.7E-04,NaN,ND
93164,93164,141.666667,38.30,2025/6/3 08:15:00,T-MG2,31,1,1.1E-03,NaN,ND


## Remap `VALUE`, `DL`, `DLV` 

We remap `DL` (Detection Limit) value to MARIS ids as follows:
    

- First check if activity (`VALUE`) is reported as "ND", based on reported detection limit `DL`:

```
if VALUE is "ND":
    if not DL: 
        VALUE, DLV, DL = NaN, NaN, 3
    else:
        VALUE, DLV, DL = DL, DL, 2
```

- Then if activity (`VALUE`) is reported:
  
```
if VALUE:
    VALUE, DLV, DL = VALUE, DL, 1
```

but if not reported, then based on detection level (`DL`) reported:

```
else:
    if DL:
        VALUE, DLV, DL = DL, DL, 2
    else:
        VALUE, DLV, DL = NaN, NaN, NaN (should be dropped)
```

With 1: Detected value (=), 2: Detection limit (<), 3: Not detected (ND)
and where:

- `VALUE` is the activity reported by TEPCO
- `DL` is initially the detection limit as reported by TEPCO but later on remapped to MARIS detection level nomenclature (categorical)
- `DLV` is the detection limit value as reported by TEPCO (copied from `DL`)




In [ ]:
#| exports
class RemapVALUE_DL_DLV_CB(Callback):
    "Remap `DL`, `DLV`, `VALUE` based on TEPCO -> MARIS rules."    
    def map_all_columns(self, row):
        """Map all three columns (VALUE, DL, DLV) at once based on TEPCO rules"""
        value, dl = row['VALUE'], row['DL']
        new_value, new_dlv, new_dl = value, dl, 1
        
        if value == 'ND':
            if pd.isna(dl):
                new_value, new_dlv, new_dl = np.nan, np.nan, 3
            else:
                new_value, new_dlv, new_dl = dl, dl, 2
                
        elif pd.isna(value):
            if pd.isna(dl):
                new_value, new_dlv, new_dl = np.nan, np.nan, np.nan
            else:
                new_value, new_dlv, new_dl = dl, dl, 2
                
        return pd.Series({
            'VALUE': new_value,
            'DLV': new_dlv, 
            'DL': new_dl
        })
        
    def __call__(self, tfm):
        mapped = tfm.dfs['SEAWATER'].apply(self.map_all_columns, axis=1)
        tfm.dfs['SEAWATER'][['VALUE', 'DLV', 'DL']] = mapped
        tfm.dfs['SEAWATER']['DL'] = tfm.dfs['SEAWATER']['DL'].astype(int)
        tfm.dfs['SEAWATER']['VALUE'] = tfm.dfs['SEAWATER']['VALUE'].astype(float)

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    RemapVALUE_DL_DLV_CB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(20)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE,DLV
34750,34750,141.034444,37.431111,2011/06/03 13:55:00,T-1,33,1,1,NaN,35.0000,NaN
1313,1313,37.410000,141.030000,2013/07/13 05:15:00,T-2-1,103,1,2,NaN,19.0000,19
22249,22249,141.033611,37.415833,2012/08/30 07:05:00,T-2,103,1,2,NaN,22.0000,22.0
84944,84944,141.200000,37.416667,2014/7/1 08:30:00,T-5,1,1,1,NaN,0.3400,NaN
24936,24936,141.033611,37.415833,2018/03/23 08:05:00,T-2,31,1,2,NaN,0.4600,0.46
7499,7499,140.665556,36.506389,2015/8/19 08:09:00,T-B,33,1,2,NaN,1.1000,1.1E+00
13593,13593,141.006944,37.055556,2015/3/17 06:42:00,T-17-1,31,1,1,NaN,0.0020,NaN
42588,42588,141.034444,37.431111,2018/01/10 07:00:00,T-1,33,1,2,NaN,0.6400,0.64
64744,64744,141.046667,37.430556,2018/01/15 07:02:00,T-0-1A,1,1,2,NaN,1.6000,1.6
5728,5728,37.410000,141.030000,2016/6/29 06:00,T-2-1,29,1,2,NaN,0.6400,0.64


## Convert activity to `Bq/m3`

Earlier in the pipeline, we assigned MARIS `unit_id` = 3 (Bq/L) to TEPCO `UNIT` = `Bq/L`.
Now we need to convert the values from Bq/L to Bq/m3 by multiplying VALUE, DL, and DLV by 1000.

In [ ]:
#| exports
class ConvertToBqM3CB(Callback):
    "Convert from Bq/L to Bq/m3."    
    def __call__(self, tfm, factor=1000):
        tfm.dfs['SEAWATER']['VALUE'] = tfm.dfs['SEAWATER']['VALUE'] * factor
        # Convert DLV to float, handling NaN values
        tfm.dfs['SEAWATER']['DLV'] = pd.to_numeric(tfm.dfs['SEAWATER']['DLV'], errors='coerce')
        tfm.dfs['SEAWATER']['DLV'] = tfm.dfs['SEAWATER']['DLV'] * factor

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    RemapVALUE_DL_DLV_CB(),
    ConvertToBqM3CB()
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(20)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE,DLV
8177,8177,140.702222,35.987500,2011/9/13 14:15:00,T-D,29,1,2,NaN,4000.0,4000.0
42106,42106,141.034444,37.431111,2017/08/15 07:09:00,T-1,29,1,2,NaN,580.0,580.0
29400,29400,141.033611,37.415833,2021/08/18 07:40:00,T-2,31,1,2,NaN,810.0,810.0
32510,32510,141.033611,37.415833,2024/02/18 07:00:00,T-2,103,1,1,NaN,14000.0,NaN
39417,39417,141.034444,37.431111,2015/06/03 07:20:00,T-1,29,1,2,NaN,600.0,600.0
71011,71011,141.062500,37.552778,2013/1/19 09:08:00,T-14,33,1,1,NaN,75.0,NaN
11388,11388,140.922222,36.905556,2011/6/3 06:15:00,T-18,29,1,2,NaN,5500.0,5500.0
11972,11972,141.000000,36.966667,2011/8/20 05:25:00,T-20,29,1,2,NaN,9000.0,9000.0
89165,89165,141.283333,38.333333,2014/4/15 10:26:00,T-MG1,33,1,1,NaN,8.0,NaN
84210,84210,141.200000,37.416667,2011/5/19 08:30:00,T-5,29,1,3,NaN,NaN,NaN


## Parse & encode time

In [ ]:
#| exports
class ParseTimeCB(Callback):
    "Parse time column from TEPCO."
    def __init__(self, time_name='TIME'): fc.store_attr()
    def __call__(self, tfm):
        tfm.dfs['SEAWATER'][self.time_name] = pd.to_datetime(tfm.dfs['SEAWATER'][self.time_name], 
                                                             format='%Y/%m/%d %H:%M:%S', errors='coerce')

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    RemapVALUE_DL_DLV_CB(),
    ConvertToBqM3CB(),
    ParseTimeCB(),
    EncodeTimeCB(),
    
    ])

df_test = tfm()['SEAWATER'] 
df_test.sample(5)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE,DLV
8169,8169,140.702222,35.987500,1312896480,T-D,31,1,2,NaN,13000.0,13000.0
73159,73159,141.072167,37.333333,1466497860,T-D9,31,1,1,NaN,2.4,NaN
65833,65833,141.046667,37.430556,1681715400,T-0-1A,103,1,1,NaN,14000.0,NaN
84693,84693,141.200000,37.416667,1360919580,T-5,31,1,1,NaN,5.2,NaN
60865,60865,141.046667,37.416111,1721029500,T-0-3A,31,1,2,NaN,390.0,390.0


## Sanitize coordinates

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    RemapVALUE_DL_DLV_CB(),
    ConvertToBqM3CB(),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

df_test = tfm()['SEAWATER']
df_test.sample(5)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE,DLV
34332,34332,141.034444,37.431111,1301301600,T-1,29,1,1,NaN,33000000.0,57000.0
34971,34971,141.034444,37.431111,1310983800,T-1,33,1,2,NaN,25000.0,25000.0
92195,92195,141.666667,38.300000,1330417560,T-MG2,31,1,2,NaN,1300.0,1300.0
52386,52386,141.040278,37.416111,1658132100,T-0-3,103,1,2,NaN,15000.0,15000.0
54149,54149,141.040278,37.430556,1526885580,T-0-1,33,1,2,NaN,530.0,530.0


## Encode to NetCDF

In [ ]:
#| eval: false
tfm = Transformer(dfs, cbs=[
    RemoveJapanaseCharCB(),
    FixRangeValueStringCB(),
    SelectColsOfInterestCB(common_coi, nuclides_pattern),
    WideToLongCB(),
    ExtractNuclideNameCB(),
    ExtractUnitCB(),
    ExtractValueTypeCB(),
    LongToWideCB(),
    RemapUnitNameCB(unit_mapping),
    RemapNuclideNameCB(nuclide_mapping),
    RemapVALUE_DL_DLV_CB(),
    ConvertToBqM3CB(),
    ParseTimeCB(),
    EncodeTimeCB(),
    SanitizeLonLatCB()
    ])

dfs_tfm = tfm()
tfm.logs

['Remove 約 (about) char',
 "Replace range values (e.g '4.0E+00<&<8.0E+00' or '1.0～2.7') by their mean",
 'Select columns of interest.',
 '\n    Get TEPCO nuclide names as values not column names \n    to extract contained information (nuclide name, unc, dl, ...).\n    ',
 'Extract nuclide name from TEPCO data.',
 'Extract unit from TEPCO data.',
 'Extract value type from TEPCO data.',
 'Reshape: long to wide',
 '\n    Remap `UNIT` name to MARIS id.\n    ',
 'Remap `NUCLIDE` name to MARIS id.',
 'Remap `DL`, `DLV`, `VALUE` based on TEPCO -> MARIS rules.',
 'Convert from Bq/L to Bq/m3.',
 'Parse time column from TEPCO.',
 'Encode time as seconds since epoch.',
 'Drop rows with invalid longitude & latitude values. Convert `,` separator to `.` separator.']

In [ ]:
dfs_tfm['SEAWATER'].sample(10)

type,SMP_ID,LON,LAT,TIME,STATION,NUCLIDE,UNIT,DL,UNC,VALUE,DLV
18167,18167,141.026389,37.322222,1402394400,T-3,31,1,1,NaN,37.0,NaN
27326,27326,141.033611,37.415833,1575270600,T-2,104,1,2,NaN,2000.0,2000.0
75081,75081,141.072222,37.416667,1415700600,T-D5,31,1,1,NaN,3.3,NaN
28056,28056,141.033611,37.415833,1594279680,T-2,33,1,2,NaN,690.0,690.0
55589,55589,141.040278,37.430556,1721112900,T-0-1,1,1,2,NaN,5500.0,5500.0
60707,60707,141.046667,37.416111,1701675960,T-0-3A,31,1,2,NaN,330.0,330.0
18458,18458,141.026389,37.322222,1463479800,T-3,33,1,1,NaN,42.0,NaN
17158,17158,141.026389,37.322222,1304065800,T-3,33,1,1,NaN,53000.0,NaN
62478,62478,141.046667,37.423333,1583134140,T-0-2,103,1,2,NaN,14000.0,14000.0
56343,56343,141.040556,37.478889,1456826100,T-6,103,1,2,NaN,17000.0,17000.0


In [ ]:
#| exports
kw = ['oceanography', 'Earth Science > Oceans > Ocean Chemistry> Radionuclides',
      'Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure',
      'Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Marine Sediments',
      'Earth Science > Oceans > Ocean Chemistry, Earth Science > Oceans > Sea Ice > Isotopes',
      'Earth Science > Oceans > Water Quality > Ocean Contaminants',
      'Earth Science > Biological Classification > Animals/Vertebrates > Fish',
      'Earth Science > Biosphere > Ecosystems > Marine Ecosystems',
      'Earth Science > Biological Classification > Animals/Invertebrates > Mollusks',
      'Earth Science > Biological Classification > Animals/Invertebrates > Arthropods > Crustaceans',
      'Earth Science > Biological Classification > Plants > Macroalgae (Seaweeds)']

In [ ]:
#| exports
def get_attrs(tfm, zotero_key, kw=kw):
    "Retrieve global attributes from MARIS dump."
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        TimeRangeCB(),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#| eval: false
get_attrs(tfm, zotero_key='JEV6HP5A', kw=kw)

{'geospatial_lat_min': '141.66666667',
 'geospatial_lat_max': '38.63333333',
 'geospatial_lon_min': '140.60388889',
 'geospatial_lon_max': '35.79611111',
 'geospatial_bounds': 'POLYGON ((140.60388889 35.79611111, 141.66666667 35.79611111, 141.66666667 38.63333333, 140.60388889 38.63333333, 140.60388889 35.79611111))',
 'time_coverage_start': '2011-03-21T14:30:00',
 'time_coverage_end': '2025-07-22T06:17:00',
 'id': 'JEV6HP5A',
 'title': "Readings of Sea Area Monitoring - Monitoring of sea water - Sea area close to TEPCO's Fukushima Daiichi NPS / Coastal area - Readings of Sea Area Monitoring [TEPCO]",
 'summary': '',
 'creator_name': '[{"creatorType": "author", "firstName": "", "lastName": "TEPCO - Tokyo Electric Power Company"}]',
 'keywords': 'oceanography, Earth Science > Oceans > Ocean Chemistry> Radionuclides, Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure, Earth Science > Oceans > Ocean Chemistry > Ocean Tracers, Earth Science > Oceans > Mar

In [ ]:
#| exports
def encode(
    fname_out: str, # Path to the folder where the NetCDF output will be saved
    **kwargs # Additional keyword arguments
    ):
    "Encode TEPCO data to NetCDF."
    dfs = load_data(fname_coastal_water, fname_clos1F, fname_iaea_orbs)
    
    tfm = Transformer(dfs, cbs=[
        RemoveJapanaseCharCB(),
        FixRangeValueStringCB(),
        SelectColsOfInterestCB(common_coi, nuclides_pattern),
        WideToLongCB(),
        ExtractNuclideNameCB(),
        ExtractUnitCB(),
        ExtractValueTypeCB(),
        LongToWideCB(),
        RemapUnitNameCB(unit_mapping),
        RemapNuclideNameCB(nuclide_mapping),
        RemapVALUE_DL_DLV_CB(),
        ConvertToBqM3CB(),
        ParseTimeCB(),
        EncodeTimeCB(),
        SanitizeLonLatCB()
    ])        
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            dest_fname=fname_out, 
                            global_attrs=get_attrs(tfm, zotero_key='JEV6HP5A', kw=kw),
                            verbose=kwargs.get('verbose', False)
                            )
    encoder.encode()

In [ ]:
#| eval: false
encode(fname_out, verbose=False)

100%|██████████| 11/11 [00:05<00:00,  2.12it/s]


In [ ]:
#| eval: false
decode(fname_in=fname_out, verbose=True)

Saved SEAWATER to ../../_data/output/tepco_SEAWATER.csv


In [ ]:
#| eval: false
df_output = pd.read_csv("../../_data/output/tepco_SEAWATER.csv")
df_output.head()

,longitude,latitude,begperiod,station,samplabcode,nuclide_id,activity,unit_id,uncertaint,detection,detection_val,samptype_id,ref_id
0,140.60388,36.29972,2011-10-13 13:21:00,T-C,5981,29,4000.0,1,NaN,<,4000.0,1,679
1,140.60388,36.29972,2011-10-13 13:21:00,T-C,5982,31,6000.0,1,NaN,<,6000.0,1,679
2,140.60388,36.29972,2011-10-13 13:21:00,T-C,5983,33,9000.0,1,NaN,<,9000.0,1,679
3,140.60388,36.29972,2011-10-13 13:23:00,T-C,5984,29,4000.0,1,NaN,<,4000.0,1,679
4,140.60388,36.29972,2011-10-13 13:23:00,T-C,5985,31,6000.0,1,NaN,<,6000.0,1,679
